[View in Colaboratory](https://colab.research.google.com/github/fatmas1982/fh2017/blob/master/Copy_of_fetch_20newsgroups5.ipynb)

In [2]:
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot') 
import numpy as np
import scipy.stats.stats as st
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#from stemming.porter2 import stem
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv

import re
from nltk import word_tokenize
import string
!pip install gensim
import  gensim.models as md
from gensim.models.phrases import Phrases, Phraser


    100% |████████████████████████████████| 22.6MB 1.7MB/s 
    100% |████████████████████████████████| 1.4MB 15.0MB/s 
    100% |████████████████████████████████| 133kB 23.2MB/s 
    100% |████████████████████████████████| 61kB 19.0MB/s 
    100% |████████████████████████████████| 4.2MB 8.6MB/s 
    100% |████████████████████████████████| 552kB 23.2MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [4]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [7]:
#https://colab.research.google.com/notebook#fileId=1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q&scrollTo=c99EvWo1s9-x
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!mkdir -p drive_PH
!google-drive-ocamlfuse drive_PH

In [26]:
!ls 'drive_PH/Colab Notebooks'

Copy of colab_topicmodeling.ipynb		  NYT_output
Copy of fetch_20newsgroups.ipynb		  pragraph_index_20.csv
Copy of new-york-times-articles (9b3b30f2).ipynb  pragraph_index.csv
Copy of new-york-times-articles.ipynb		  Sentences_20.csv
fetch_20newsgroups.ipynb			  Sentences.csv
input						  Sentences_not_stops_20.csv
lesks.csv					  Sentences_not_stops.csv


In [0]:
path_database='./drive_PH/Colab Notebooks/' 
path_stop_word=path_database+'input/stopwords/'

pragraph_index='pragraph_index_20.csv'
Sentences='Sentences_20.csv'
Sentences_not_stops='Sentences_not_stops_20.csv'
lesk='lesk_20.csv'

In [41]:
import os
os.listdir(path_stop_word)

['stopwords.txt']

In [10]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [11]:
documents[0]
dataset.filenames

array(['/content/scikit_learn_data/20news_home/20news-bydate-train/talk.politics.mideast/76141',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/alt.atheism/53281',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/talk.politics.mideast/76350',
       ...,
       '/content/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/105105',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.mac.hardware/51575',
       '/content/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/104908'],
      dtype='<U89')

In [0]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):#header_list=['index','text']
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line]#,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()

In [0]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

In [0]:
def pragraph_to_setnences(str):
    return sent_tokenize(str)

In [0]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs

In [0]:
def read_text_from_database(path_database,file_databbase):
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph

In [0]:
def stopwords_list():
    stopwordsFile = open(path_stop_word+'stopwords.txt')
    stopwordsFile.seek(0)
    stopwordsV1 = stopwordsFile.readlines()
    stopwordsV2 = []
    for sent in stopwordsV1:
        sent.replace('\n', '')
        new_word = sent[0:len(sent) - 1]
        stopwordsV2.append(new_word.lower())
    return stopwordsV2

In [0]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
stopwordsV2=stopwords_list()
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(str):
   
            
    list_word=[]
    tokenizer = RegexpTokenizer("[\w']+")
    
    words=tokenizer.tokenize(str)
    for word in words:
        new_word = word.encode('ascii', 'ignore').decode('utf-8')
        if new_word != '':
    
            english_stops = set(stopwords.words('english'))
           
            list_word=[new_word for new_word in words if new_word.lower() not in english_stops
                       and new_word.lower() not in new_stop_words 
                       and new_word.lower() not in new_stop_words2 
                       and  not new_word.lower().isdigit() 
                       and new_word.lower() not in digits 
                       and new_word.lower() not in  numbers and word.lower() not in stopwordsV2
                       and new_word.lower() not in string.punctuation]
    
  
    
    return list_word#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

In [0]:
import nltk
nltk.download('punkt')
def paragraphs_to_sentece(pragraphs):
    sentenses_list=[]
    
    for index_p in  range(len(pragraphs)):
        #print(index_p)
        #print("pppppppppppppppppppp")
        #print(pragraphs[index_p])
        if pragraphs[index_p] is not None:
          #print("setnences")
          #p1=txt_pragraphs(pragraphs[index_p])
          setnences=pragraph_to_setnences(str(pragraphs[index_p]))
          #print("sssssssssssssssssssssssssss")
          #print("setnences")#,setnences)

          for indexs in range(len(setnences)):
              row=[]
              #print(setnences)
              row.append(index_p)
              row.append(indexs)
              row.append(setnences[indexs])
              sentenses_list.append(row)
    header_list=['index_P','index_sent','sentence']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#paragraphs_to_sentece(paragraphs.text)

In [0]:
!ls nytimes_news_articles.txt

In [0]:
nltk.download('stopwords')
def  sentece_Not_stop_word(setences):
    #words_list=[]
    sentenses_list=[]
    
    for index_s in  range(len(setences)):
            
          #print("Sentence No. ",index_s,": ",setences.loc[index_s]['sentence'],"\n")
          words=remove_stopword_sentences(str(setences.loc[index_s]['sentence']))
          wordsent=''
          row=[]
          

          row.append(setences.loc[index_s]['index_P'])
          row.append(setences.loc[index_s]['index_sent'])
          row.append(words)
          sentenses_list.append(row)
    header_list=['index_P','index_sent','words_not_stop']
    df = pd.DataFrame(sentenses_list, columns=header_list)#, index=index)
    #df

    #print(sentenses_list)
    df.to_csv(path_database+Sentences_not_stops, encoding='utf-8', index=False)
    #save_file_to_database(sentenses_list,path_database,"Sentences.csv",header_list)        
    return df

#sentece_Not_stop_word(setences)

In [0]:
import pandas as pd
def courps_to_CSV_docs():
    #Reading the news articles file
    nyTimesFile = open('./nytimes_news_articles.txt', encoding='latin-1')
    nyTimesFile.seek(0)
    nyTimesV1 = nyTimesFile.readlines()
    nyTimesTemp = []
    nyTimesURL = []

    for i in range(0, len(nyTimesV1)-1):
        if re.findall('URL', nyTimesV1[i]) == []:
            sent = sent + nyTimesV1[i]
            if (re.findall('URL', nyTimesV1[i+1]) != []) and (i+1 < len(nyTimesV1)):
                nyTimesTemp.append(sent.strip())
        else:
            sent = ''
            nyTimesURL.append(nyTimesV1[i])

    for i in range(0, len(nyTimesTemp)):
        nyTimesTemp[i] = nyTimesTemp[i]+'articleID'+str(i)
    print(len(nyTimesTemp))
    header_list=['index','text']
    save_file_to_database(nyTimesTemp,path_database,pragraph_index,header_list)
    '''for i in range(1):
        print(i,"============================================")
        print("============================================")'''
    #nytimes = preProcessor(nyTimesTemp)
    print("============================================")
    #print(nytimes)

In [0]:
header_list=['index','text']
#save_file_to_database(documents,'/',pragraph_index,header_list)

In [0]:
#save_file_to_database(documents,path_database,pragraph_index,header_list)

In [0]:
#courps_to_CSV_docs()

In [0]:
paragraphs=read_cvs_by_pands(path_database,pragraph_index,None,0)
paragraphs

In [0]:
!ls

In [55]:
setences=read_cvs_by_pands(path_database,Sentences,None,0)
setences

,index_P,index_sent,sentence
0,0,0.0,Well i'm not sure about the story nad it did s...
1,0,1.0,What\nI disagree with is your statement that t...
2,0,2.0,That is rediculous.
3,0,3.0,The U.S. media is\nthe most pro-israeli media ...
4,0,4.0,Having lived in Europe\nI realize that inciden...
5,0,5.0,The U.S. media as a whole seem to try to\nigno...
6,0,6.0,The U.S. is subsidizing Israels existance and ...
7,0,7.0,So I think\nthat might be a reason they report...
8,0,8.0,"What is a shame is that in Austria, daily repo..."
9,0,9.0,"After all, look how the Jews are treating othe..."


In [56]:
df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
df_Sentences_not_stops

,index_P,index_sent,words_not_stop
0,0,0.0,"['Well', ""i'm"", 'sure', 'story', 'nad', 'seem'..."
1,0,1.0,"['disagree', 'statement', 'U', 'Media', 'ruin'..."
2,0,2.0,['rediculous']
3,0,3.0,"['U', 'media', 'pro', 'israeli', 'media', 'wor..."
4,0,4.0,"['lived', 'Europe', 'realize', 'incidences', '..."
5,0,5.0,[]
6,0,6.0,"['U', 'subsidizing', 'Israels', 'existance', '..."
7,0,7.0,"['think', 'might', 'reason', 'report', 'clearl..."
8,0,8.0,[]
9,0,9.0,"['look', 'Jews', 'treating', 'races', 'got', '..."


In [0]:
nltk.download('wordnet')

In [0]:
'''
this function for compute lesk of word in sentence
'''
def lesk_word_sentence(sentence,words):
    from nltk.wsd import lesk
    lesk_synset=''
    
    lesks_row= []
    lesks_name=[]
    #print(type(words))
    for i in range(len(words)):
        
        disambiguated=lesk(context_sentence=str(sentence), ambiguous_word=words[i].strip())
        
        
        if disambiguated is not None:
            lesks_row.append(disambiguated)
            lesks_name.append(disambiguated.name())
    
    
    return lesks_row,lesks_name
#disambiguated#lesk_synset

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#ll=['Well','"im"','sure','story','nad','seem','biased']
#lesk_word_sentence("Well i'm not sure about the story nad it did seem biased.", ll)
#lesk('I love dog', 'dog')



In [201]:


def sent_lesks():
    df_Sentences=read_cvs_by_pands(path_database,Sentences,None,0)
    #print(df_Sentences.columns)
    df_Sentences_not_stops=read_cvs_by_pands(path_database,Sentences_not_stops,None,0)
    lesks=[]
    for i in range(len(df_Sentences_not_stops)):
        sent=[]
        full_words=df_Sentences_not_stops['words_not_stop'][i].replace("[",'').replace("]",'').replace("'",'').replace('"','')
        full_words_list=full_words.replace(",",'-')      
        words=full_words.split(',')
        sentense=df_Sentences['sentence'][i]
        
        sent.append(df_Sentences['index_P'][i])
        sent.append(df_Sentences['index_sent'][i])
        ss=lesk_word_sentence(sentense, words)
        
        sent.append(full_words_list)
        sent.append(ss[0])
        sent.append(ss[1])
        
        lesks.append(sent)
        
    header_list=['index_P','index_sent','full_word','lesks','lesks_name']
    df = pd.DataFrame(lesks, columns=header_list)
    
    df.to_csv(path_database+'lesk_20.csv', encoding='utf-8', index=False)  
    return df
    #return lesks
sent_lesks()
    

,index_P,index_sent,full_word,lesks,lesks_name
0,0,0.0,Well- im- sure- story- nad- seem- biased,"[Synset('well.v.01'), Synset('surely.r.01'), S...","[well.v.01, surely.r.01, story.n.02, nicotinam..."
1,0,1.0,disagree- statement- U- Media- ruin- Israels- ...,"[Synset('disagree.v.02'), Synset('statement.n....","[disagree.v.02, statement.n.07, uranium.n.01, ..."
2,0,2.0,rediculous,[],[]
3,0,3.0,U- media- pro- israeli- media- world,"[Synset('uranium.n.01'), Synset('medium.n.08')...","[uranium.n.01, medium.n.08, pro.r.01, israeli...."
4,0,4.0,lived- Europe- realize- incidences- one- descr...,"[Synset('live.v.07'), Synset('europe.n.01'), S...","[live.v.07, europe.n.01, realize.v.06, inciden..."
5,0,5.0,,[],[]
6,0,6.0,U- subsidizing- Israels- existance- Europeans-...,"[Synset('uranium.n.01'), Synset('subsidize.v.0...","[uranium.n.01, subsidize.v.02, israel.n.02, eu..."
7,0,7.0,think- might- reason- report- clearly- atrocities,"[Synset('think.v.13'), Synset('might.n.01'), S...","[think.v.13, might.n.01, reason.v.01, reputati..."
8,0,8.0,,[],[]
9,0,9.0,look- Jews- treating- races- got- power,"[Synset('spirit.n.02'), Synset('jew.n.01'), Sy...","[spirit.n.02, jew.n.01, treat.v.08, subspecies..."


In [203]:
l=read_cvs_by_pands(path_database,'lesk_20.csv',None,0)
l

,index_P,index_sent,full_word,lesks,lesks_name
0,0,0.0,Well- im- sure- story- nad- seem- biased,"[Synset('well.v.01'), Synset('surely.r.01'), S...","['well.v.01', 'surely.r.01', 'story.n.02', 'ni..."
1,0,1.0,disagree- statement- U- Media- ruin- Israels- ...,"[Synset('disagree.v.02'), Synset('statement.n....","['disagree.v.02', 'statement.n.07', 'uranium.n..."
2,0,2.0,rediculous,[],[]
3,0,3.0,U- media- pro- israeli- media- world,"[Synset('uranium.n.01'), Synset('medium.n.08')...","['uranium.n.01', 'medium.n.08', 'pro.r.01', 'i..."
4,0,4.0,lived- Europe- realize- incidences- one- descr...,"[Synset('live.v.07'), Synset('europe.n.01'), S...","['live.v.07', 'europe.n.01', 'realize.v.06', '..."
5,0,5.0,NaN,[],[]
6,0,6.0,U- subsidizing- Israels- existance- Europeans-...,"[Synset('uranium.n.01'), Synset('subsidize.v.0...","['uranium.n.01', 'subsidize.v.02', 'israel.n.0..."
7,0,7.0,think- might- reason- report- clearly- atrocities,"[Synset('think.v.13'), Synset('might.n.01'), S...","['think.v.13', 'might.n.01', 'reason.v.01', 'r..."
8,0,8.0,NaN,[],[]
9,0,9.0,look- Jews- treating- races- got- power,"[Synset('spirit.n.02'), Synset('jew.n.01'), Sy...","['spirit.n.02', 'jew.n.01', 'treat.v.08', 'sub..."


In [0]:
p=read_cvs_by_pands(path_database,'pragraph_index_20.csv',None,0)
index_p_p=p['index']


In [316]:
l_names=[]
for i in range(len(index_p_p)) :
    l_names=l[l['index_P']==str(i)]['lesks_name']
print(l_names)

128385                  ['controversy.n.01', 'potato.n.01']
128386                                                   []
128387                                                   []
128388    ['look.v.02', 'viable.s.02', 'frame-up.n.01', ...
128389                                                   []
128390                                   ['concerned.s.02']
128391                                                   []
128392    ['have_a_bun_in_the_oven.v.01', 'total.v.01', ...
128393                                                   []
128394    ['strait.n.01', 'like.n.02', 'old.s.07', 'road...
128395                     ['reasonably.r.01', 'pall.v.01']
128396                    ['release.n.08', 'pullover.n.01']
128397    ['ill.r.01', 'check_mark.n.01', 'uniform.n.01'...
128398                                                   []
128399    ['receive.v.02', 'pullover.n.01', 'shirt.v.01'...
128400    ['visit.v.01', 'might.n.01', 'wish.n.01', 'var...
128401                                  

In [321]:
len(index_p_p)
l_names=l[l['index_P']==str(11313)]['lesks_name']
l_names

128385                  ['controversy.n.01', 'potato.n.01']
128386                                                   []
128387                                                   []
128388    ['look.v.02', 'viable.s.02', 'frame-up.n.01', ...
128389                                                   []
128390                                   ['concerned.s.02']
128391                                                   []
128392    ['have_a_bun_in_the_oven.v.01', 'total.v.01', ...
128393                                                   []
128394    ['strait.n.01', 'like.n.02', 'old.s.07', 'road...
128395                     ['reasonably.r.01', 'pall.v.01']
128396                    ['release.n.08', 'pullover.n.01']
128397    ['ill.r.01', 'check_mark.n.01', 'uniform.n.01'...
128398                                                   []
128399    ['receive.v.02', 'pullover.n.01', 'shirt.v.01'...
128400    ['visit.v.01', 'might.n.01', 'wish.n.01', 'var...
128401                                  

In [340]:
def isNotBlank (myString):
    if myString and myString.strip():
        #myString is not None AND myString is not empty or blank
        return True
    #myString is None OR myString is empty or blank
    return False
isNotBlank(' ')

False

In [0]:
def paragarph_to_lesk():
    pragraph_list=[]
    for p in range(len(index_p_p)) :
        #print(p)
        l_names=l[l['index_P']==str(p)]['lesks_name']
        #print(len(l_names))

        l_name_one_paragraph=''
        paragraph_row=[]
        for i in range(len(l_names)):
            l_name_one_paragraph+=l_names.get_values()[i].replace('[','').replace(']',',').strip()

        if isNotBlank(l_name_one_paragraph.replace(',','')):


            paragraph_row.append(p)
            paragraph_row.append(l_name_one_paragraph)
            pragraph_list.append(paragraph_row)
        else:
            print(p,"pppppppppppppp")
    #print(pragraph_list)
    header_list=['index_P','lesk_list']
    df = pd.DataFrame(pragraph_list,columns=header_list)
    df.to_csv(path_database+'paragraph_lesk_20.csv', encoding='utf-8', index=False)  
    return df
    #df

In [365]:
p_lesk=read_cvs_by_pands(path_database,'paragraph_lesk_20.csv',None,0)
#p_lesk[p_lesk['lesk_list']=='[]']
p_lesk['lesk_list'].applay(replace("'",''))

AttributeError: ignored

In [388]:
document_0 = "China has a strong economy that is growing at a rapid pace. However politically it differs greatly from the US Economy."
document_1 = "At last, China seems serious about confronting an endemic problem: domestic violence and corruption."
document_2 = "Japan's prime minister, Shinzo Abe, is working towards healing the economic turmoil in his own country for his view on the future of his people."
document_3 = "Vladimir Putin is working hard to fix the economy in Russia as the Ruble has tumbled."
document_4 = "What's the future of Abenomics? We asked Shinzo Abe for his views"
document_5 = "Obama has eased sanctions on Cuba while accelerating those against the Russian Economy, even as the Ruble's value falls almost daily."
document_6 = "Vladimir Putin is riding a horse while hunting deer. Vladimir Putin always seems so serious about things - even riding horses. Is he crazy?"

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]
all_documents
#in Scikit-Learn
tokenize = lambda doc: doc.lower().split(" ")
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(all_documents)
sklearn_representation.data
sklearn_tfidf.get_feature_names

<bound method CountVectorizer.get_feature_names of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=False,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7f4e6dc3e620>, use_idf=True,
        vocabulary=None)>

In [392]:
print(p_lesk['lesk_list'][1:10])

1    'yea.r.01', 'expect.v.05', 'people.n.03', 'und...
2    'realize.v.06', 'principle.n.04', 'one.s.06', ...
3    ,'last.v.01', 'product.n.05', 'sphere.n.01', '...
4    'well.v.01', 'variety.n.06', 'score.v.06', 'pl...
5    ,'look.v.02', 'sidereal_day.n.01', 'stay.v.09'...
6    'very_well.r.02','record.v.01', 'show.v.12', '...
7    'voice.v.02', 'wish.v.02', 'desirous.a.01', 't...
8    'cipher.n.04', 'suppose.v.01', 'let.v.01', 'us...
9    'wonder.v.01', 'shed.n.01', 'sparkle.n.01', 'e...
Name: lesk_list, dtype: object


In [0]:
cv_tfidf_lesk = TfidfVectorizer(analyzer='word',token_pattern="'"+'(?u)\\b\\w\\w+\\b\\.\\w\\.\\d\\d'+"'") #lesk
#print(type(texts_lesk))
cv_tfidf_fit_lesk=cv_tfidf_lesk.fit_transform(p_lesk['lesk_list']).toarray()
df_tfidf_lesk=pd.DataFrame(cv_tfidf_fit_lesk,columns=cv_tfidf_lesk.get_feature_names(),index=p_lesk['index_P'])
df_tfidf_lesk.to_csv(path_database+'tf_idf_lesk_table_20.csv')
#save_file_to_database(texts_lesk,path_database,lesk_paragraph,lesk_paragraph_list)
df_tfidf_lesk